# SK-4-Filters : Filtres et Observabilite

**Navigation** : [<< 03-Agents](03-SemanticKernel-Agents.ipynb) | [Index](README.md) | [05-VectorStores >>](05-SemanticKernel-VectorStores.ipynb)

---

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Intercepter les appels de **fonctions** avec des filtres
2. Modifier les **prompts** avant envoi au LLM
3. Controler le **Function Calling automatique**
4. Configurer le **logging** pour le debugging
5. Comprendre l'integration **OpenTelemetry** pour le monitoring

### Prerequis

- Python 3.10+
- Notebooks 01-03 completes
- Cle API OpenAI configuree (`.env`)

### Duree estimee : 45 minutes

---

## Sommaire

| Section | Contenu | Concepts cles |
|---------|---------|---------------|
| 1 | Introduction | Pourquoi les filtres ? |
| 2 | Function Filters | Avant/apres invocation |
| 3 | Prompt Filters | Modification du prompt |
| 4 | Auto-Invoke Filters | Controle du function calling |
| 5 | Logging | Configuration, niveaux |
| 6 | OpenTelemetry | Tracing, metriques |
| 7 | Conclusion | Resume, exercices |

> **Pourquoi les filtres ?** Les filtres permettent d'intercepter et modifier les appels a tous les niveaux de SK : avant/apres les fonctions, avant/apres les prompts, et lors du function calling automatique. C'est essentiel pour le logging, la securite, et le monitoring en production.

In [1]:
# Installation et imports
%pip install semantic-kernel python-dotenv --quiet

import os
import logging
from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function, KernelArguments
from semantic_kernel.filters import FilterTypes
from semantic_kernel.exceptions import KernelInvokeException

# Chargement du fichier .env (cles API)
load_dotenv("../.env")

api_key = os.getenv("OPENAI_API_KEY")
print(f"Configuration: API Key {'OK' if api_key else 'MANQUANTE'}")
print("Imports OK")


[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: C:\Users\jsboi\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


Configuration: API Key OK
Imports OK


## 1. Introduction aux Filtres

Semantic Kernel propose un systeme de filtres inspire des middlewares web :

| Type de Filtre | Point d'interception | Cas d'usage |
|----------------|---------------------|-------------|
| **Function Invocation** | Avant/apres chaque fonction | Logging, validation, timing |
| **Prompt Rendering** | Avant envoi au LLM | Injection de regles, anonymisation |
| **Auto Function Invocation** | Lors du function calling | Rate limiting, approbation |

### Architecture des filtres

```
User Request
    |
    v
[Prompt Filter] --> Modifie le prompt
    |
    v
LLM Call
    |
    v
[Auto-Invoke Filter] --> Controle les appels de fonction
    |
    v
[Function Filter] --> Avant/apres chaque fonction
    |
    v
Response
```

## 2. Function Invocation Filters

Les filtres de fonction permettent d'intercepter chaque appel de fonction du kernel.

In [2]:
from semantic_kernel.filters.functions.function_invocation_context import FunctionInvocationContext
from typing import Callable, Coroutine, Any
import time

# Creation du kernel
kernel = Kernel()
kernel.add_service(OpenAIChatCompletion(service_id="default"))

# Plugin de demonstration
class MathPlugin:
    @kernel_function(description="Additionne deux nombres")
    def add(self, a: int, b: int) -> int:
        return a + b
    
    @kernel_function(description="Multiplie deux nombres")
    def multiply(self, a: int, b: int) -> int:
        return a * b

kernel.add_plugin(MathPlugin(), plugin_name="math")

# Filtre de logging avec timing
@kernel.filter(FilterTypes.FUNCTION_INVOCATION)
async def logging_filter(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Coroutine[Any, Any, None]]
):
    """Filtre qui log les appels de fonction avec leur duree."""
    func_name = f"{context.function.plugin_name}.{context.function.name}"
    print(f"[AVANT] Appel de {func_name}")
    print(f"  Arguments: {context.arguments}")
    
    start_time = time.time()
    
    # Appel de la fonction (ou du filtre suivant)
    await next(context)
    
    duration = time.time() - start_time
    print(f"[APRES] {func_name} termine en {duration:.4f}s")
    print(f"  Resultat: {context.result}")

# Test du filtre
result = await kernel.invoke(kernel.get_function("math", "add"), KernelArguments(a=5, b=3))
print(f"\nResultat final: {result}")

[AVANT] Appel de math.add
  Arguments: {'a': 5, 'b': 3}
[APRES] math.add termine en 0.0001s
  Resultat: 8

Resultat final: 8


### Interpretation : Function Invocation Filter

Le filtre ci-dessus illustre le pattern **middleware** :

1. **Avant l'appel** : On log le nom de la fonction et ses arguments
2. **`await next(context)`** : On execute la fonction (ou le filtre suivant)
3. **Apres l'appel** : On log le resultat et la duree

**Points cles** :
- `context.function` : Metadata de la fonction (nom, plugin, description)
- `context.arguments` : Arguments passes a la fonction
- `context.result` : Resultat apres execution
- `next(context)` : Appelle le filtre suivant ou la fonction elle-meme

> **Attention** : Oublier `await next(context)` bloquera l'execution de la fonction !

In [3]:
# Exemple de filtre de validation
@kernel.filter(FilterTypes.FUNCTION_INVOCATION)
async def validation_filter(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Coroutine[Any, Any, None]]
):
    """Filtre qui valide les arguments avant execution."""
    # Validation specifique pour les fonctions math
    if context.function.plugin_name == "math":
        a = context.arguments.get("a", 0)
        b = context.arguments.get("b", 0)
        
        # Exemple : bloquer les nombres negatifs
        if a < 0 or b < 0:
            raise ValueError(f"Les nombres negatifs ne sont pas autorises: a={a}, b={b}")
    
    await next(context)

# Test avec nombres valides
try:
    result = await kernel.invoke(kernel.get_function("math", "multiply"), KernelArguments(a=4, b=5))
    print(f"Resultat: {result}")
except Exception as e:
    print(f"Erreur: {e}")

# Test avec nombre negatif (devrait echouer)
try:
    result = await kernel.invoke(kernel.get_function("math", "multiply"), KernelArguments(a=-3, b=5))
    print(f"Resultat: {result}")
except KernelInvokeException as e:
    # L'exception ValueError est encapsulee dans KernelInvokeException
    print(f"Erreur de validation (KernelInvokeException): nombres negatifs bloques")
except ValueError as e:
    print(f"Erreur de validation: {e}")

Function failed. Error: Les nombres negatifs ne sont pas autorises: a=-3, b=5


Something went wrong in function invocation. During function invocation: 'math-multiply'. Error description: 'Les nombres negatifs ne sont pas autorises: a=-3, b=5'


[AVANT] Appel de math.multiply
  Arguments: {'a': 4, 'b': 5}
[APRES] math.multiply termine en 0.0000s
  Resultat: 20
Resultat: 20
[AVANT] Appel de math.multiply
  Arguments: {'a': -3, 'b': 5}
Erreur de validation (KernelInvokeException): nombres negatifs bloques


### Interprétation : Validation Filter avec Exception Handling

**Sortie obtenue** : 
- Premier appel (4 × 5) : Succès avec résultat 20
- Second appel (-3 × 5) : Blocage avec KernelInvokeException

| Aspect | Comportement | Explication |
|--------|--------------|-------------|
| **Exception wrapping** | ValueError → KernelInvokeException | SK encapsule les exceptions des filtres |
| **Ordre d'exécution** | Validation → Logging → Fonction | Les filtres sont chaînés dans l'ordre d'enregistrement |
| **Blocage précoce** | Avant calcul | Le filtre de validation empêche l'exécution de la fonction |

**Points clés** :
1. **Chaînage des filtres** : Le filtre de validation s'exécute AVANT le filtre de logging (voir les logs)
2. **Exception handling** : Les exceptions levées dans les filtres sont automatiquement wrappées dans `KernelInvokeException`
3. **Validation métier** : Pattern idéal pour valider les inputs avant traitement coûteux (LLM, DB, etc.)
4. **Court-circuit** : Si on lève une exception, `next(context)` n'est jamais appelé → fonction non exécutée

**Note technique** : Pour capturer l'exception originale, utiliser `try/except KernelInvokeException as e` et inspecter `e.__cause__`.

## 3. Prompt Rendering Filters

Les filtres de prompt permettent de modifier le prompt avant son envoi au LLM.

In [4]:
from semantic_kernel.filters.prompts.prompt_render_context import PromptRenderContext
from semantic_kernel.prompt_template import PromptTemplateConfig

# Nouveau kernel pour les filtres de prompt
kernel_prompt = Kernel()
kernel_prompt.add_service(OpenAIChatCompletion(service_id="default"))

# Filtre qui ajoute des instructions de securite
@kernel_prompt.filter(FilterTypes.PROMPT_RENDERING)
async def security_prompt_filter(
    context: PromptRenderContext,
    next: Callable[[PromptRenderContext], Coroutine[Any, Any, None]]
):
    """Ajoute des regles de securite au prompt."""
    # Executer le rendu du template d'abord
    await next(context)
    
    # Ajouter des instructions de securite apres le rendu
    security_rules = """

REGLES DE SECURITE:
- Ne jamais reveler d'informations personnelles
- Ne pas generer de contenu offensant
- Refuser les demandes de code malveillant
"""
    context.rendered_prompt = context.rendered_prompt + security_rules
    print(f"[Prompt Filter] Regles de securite ajoutees")

# Creation d'une fonction avec template
prompt_config = PromptTemplateConfig(
    template="Tu es un assistant. Reponds a: {{$input}}",
    name="secure_chat",
    template_format="semantic-kernel"
)

chat_function = kernel_prompt.add_function(
    function_name="chat",
    plugin_name="demo",
    prompt_template_config=prompt_config
)

# Test
response = await kernel_prompt.invoke(chat_function, KernelArguments(input="Bonjour, comment ca va?"))
print(f"\nReponse: {response}")

[Prompt Filter] Regles de securite ajoutees



Reponse: Bonjour! Je vais bien, merci. Comment puis-je vous aider aujourd'hui?


### Interprétation : Prompt Filter avec Injection de Règles

**Sortie obtenue** : Le message "[Prompt Filter] Regles de securite ajoutees" confirme que le filtre s'est exécuté, et le LLM a bien reçu le prompt modifié.

| Étape | Contenu | Rôle |
|-------|---------|------|
| **Template original** | "Tu es un assistant. Reponds a: {{$input}}" | Défini par le développeur |
| **Rendu** | "Tu es un assistant. Reponds a: Bonjour..." | Variables remplacées |
| **Après filtre** | Prompt + REGLES DE SECURITE | Instructions système ajoutées |
| **Envoyé au LLM** | Prompt complet avec règles | Le LLM "voit" les règles |

**Points clés** :
1. **Ordre d'exécution** : D'abord `await next(context)` (rendu du template), PUIS modification du `rendered_prompt`
2. **Injection transparente** : Le développeur qui appelle la fonction ne sait pas que des règles sont ajoutées
3. **Sécurité systémique** : Toutes les fonctions du kernel héritent automatiquement des règles
4. **Visibilité** : Le LLM "voit" les règles dans son prompt (contrairement aux system messages cachés)

**Cas d'usage production** :
- Ajout de contraintes légales (RGPD, conformité)
- Injection de contexte d'entreprise (brand guidelines)
- Rate limiting explicite ("Tu as droit à 3 appels de fonction max")
- Instructions de formatage (JSON, Markdown, etc.)

**Note** : Pour voir le prompt complet envoyé au LLM, activer le logging en mode DEBUG (voir section 5).

## 4. Auto Function Invocation Filters

Ces filtres controlent le comportement du function calling automatique.

In [5]:
from semantic_kernel.filters.auto_function_invocation.auto_function_invocation_context import AutoFunctionInvocationContext
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings
from semantic_kernel.connectors.ai import FunctionChoiceBehavior

# Nouveau kernel pour les filtres auto-invoke
kernel_auto = Kernel()
kernel_auto.add_service(OpenAIChatCompletion(service_id="default"))

# Plugin sensible
class DatabasePlugin:
    @kernel_function(description="Execute une requete SQL")
    def execute_sql(self, query: str) -> str:
        print(f"  [DB] Execution: {query}")
        return f"Resultats pour: {query}"
    
    @kernel_function(description="Supprime des donnees")
    def delete_data(self, table: str) -> str:
        print(f"  [DB] DELETE FROM {table}")
        return f"Donnees supprimees de {table}"

kernel_auto.add_plugin(DatabasePlugin(), plugin_name="database")

# Compteur d'appels pour rate limiting
call_count = {"total": 0}

@kernel_auto.filter(FilterTypes.AUTO_FUNCTION_INVOCATION)
async def rate_limit_filter(
    context: AutoFunctionInvocationContext,
    next: Callable[[AutoFunctionInvocationContext], Coroutine[Any, Any, None]]
):
    """Limite le nombre d'appels de fonction automatiques."""
    MAX_CALLS = 3
    
    call_count["total"] += 1
    
    if call_count["total"] > MAX_CALLS:
        print(f"[Rate Limit] Limite atteinte ({MAX_CALLS} appels max)")
        context.terminate = True  # Stoppe le function calling
        return
    
    func_name = context.function.name
    print(f"[Auto-Invoke] Appel #{call_count['total']}: {func_name}")
    
    # Bloquer les operations dangereuses
    if "delete" in func_name.lower():
        print(f"[Auto-Invoke] BLOQUE: Operation de suppression non autorisee")
        context.terminate = True
        return
    
    await next(context)

print("Filtre auto-invoke configure. Voir section 5 pour un exemple complet.")

Filtre auto-invoke configure. Voir section 5 pour un exemple complet.


### Interprétation : Auto Function Invocation Filter avec Rate Limiting

**Architecture du contrôle** : Ce filtre intercepte les décisions du LLM AVANT l'exécution des fonctions.

| Mécanisme | Implémentation | Effet |
|-----------|----------------|-------|
| **Rate limiting** | Compteur global `call_count` | Maximum 3 appels de fonction |
| **Blacklist** | Check sur `"delete" in func_name` | Bloque les opérations dangereuses |
| **Terminaison** | `context.terminate = True` | Stoppe le function calling loop |
| **Court-circuit** | Return sans `await next(context)` | Fonction non exécutée |

**Points clés** :
1. **Function calling loop** : Quand le LLM appelle une fonction, SK peut automatiquement exécuter la fonction et renvoyer le résultat au LLM (qui peut ensuite appeler une autre fonction, etc.)
2. **`terminate = True`** : Indique à SK d'arrêter le loop et de retourner la réponse actuelle du LLM
3. **Sécurité critique** : Sans ce filtre, un LLM pourrait décider d'appeler `delete_data()` sans supervision
4. **Audit trail** : Chaque tentative d'appel est loggée, même si bloquée

**Workflow complet** :
```
User: "Supprime les anciennes données"
    ↓
LLM decide: database.delete_data(table="old_data")
    ↓
[Auto-Invoke Filter] BLOQUE: Operation de suppression
    ↓
LLM reçoit: "Opération refusée"
    ↓
Response finale
```

**Note technique** : Ce filtre est complémentaire aux Function Filters (section 2). L'Auto-Invoke Filter contrôle les appels **automatiques** du LLM, tandis que les Function Filters interceptent **tous** les appels (manuels ou auto).

## 5. Logging et Debugging

SK utilise le module `logging` standard de Python.

In [6]:
import logging
import sys

# Configuration du logging pour SK
def configure_sk_logging(level=logging.INFO):
    """Configure le logging pour Semantic Kernel."""
    # Handler pour la console
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(level)
    
    # Format detaille
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    handler.setFormatter(formatter)
    
    # Configurer le logger SK
    sk_logger = logging.getLogger("semantic_kernel")
    sk_logger.setLevel(level)
    sk_logger.addHandler(handler)
    
    return sk_logger

# Niveaux de logging disponibles
print("Niveaux de logging disponibles:")
print("| Niveau   | Valeur | Description                              |")
print("|----------|--------|------------------------------------------|")
print("| DEBUG    | 10     | Details tres fins (prompts complets)     |")
print("| INFO     | 20     | Informations generales                   |")
print("| WARNING  | 30     | Avertissements                           |")
print("| ERROR    | 40     | Erreurs                                  |")
print("| CRITICAL | 50     | Erreurs critiques                        |")

# Activer le logging DEBUG pour voir les details
logger = configure_sk_logging(logging.DEBUG)
print("\nLogging configure en mode DEBUG")

Niveaux de logging disponibles:
| Niveau   | Valeur | Description                              |
|----------|--------|------------------------------------------|
| DEBUG    | 10     | Details tres fins (prompts complets)     |
| INFO     | 20     | Informations generales                   |
| WARNING  | 30     | Avertissements                           |
| ERROR    | 40     | Erreurs                                  |
| CRITICAL | 50     | Erreurs critiques                        |

Logging configure en mode DEBUG


In [7]:
# Exemple avec logging actif
from semantic_kernel.prompt_template import PromptTemplateConfig

kernel_log = Kernel()
kernel_log.add_service(OpenAIChatCompletion(service_id="default"))

# Fonction simple
simple_config = PromptTemplateConfig(
    template="Dis bonjour a {{$name}} en francais.",
    name="greet"
)

greet_function = kernel_log.add_function(
    function_name="greet",
    plugin_name="demo",
    prompt_template_config=simple_config
)

# Execution avec logging
print("=" * 50)
print("Execution avec logging actif:")
print("=" * 50)
response = await kernel_log.invoke(greet_function, KernelArguments(name="Alice"))
print(f"\nReponse finale: {response}")

2026-02-04 08:07:55,813 - semantic_kernel.prompt_template.kernel_prompt_template - DEBUG - Extracting blocks from template: Dis bonjour a {{$name}} en francais.


Execution avec logging actif:
2026-02-04 08:07:55,814 - semantic_kernel.functions.kernel_function - INFO - Function demo-greet invoking.


2026-02-04 08:07:55,815 - semantic_kernel.functions.kernel_function - DEBUG - Function arguments: {'name': 'Alice'}


2026-02-04 08:07:55,815 - semantic_kernel.prompt_template.kernel_prompt_template - DEBUG - Rendering list of 3 blocks


2026-02-04 08:07:55,816 - semantic_kernel.prompt_template.kernel_prompt_template - DEBUG - Rendered prompt: Dis bonjour a Alice en francais.


2026-02-04 08:07:56,214 - semantic_kernel.connectors.ai.open_ai.services.open_ai_handler - INFO - OpenAI usage: CompletionUsage(completion_tokens=3, prompt_tokens=16, total_tokens=19, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))


2026-02-04 08:07:56,215 - semantic_kernel.functions.kernel_function - INFO - Function demo-greet succeeded.


2026-02-04 08:07:56,216 - semantic_kernel.functions.kernel_function - DEBUG - Function result: Bonjour Alice !


2026-02-04 08:07:56,216 - semantic_kernel.functions.kernel_function - INFO - Function completed. Duration: 0.400880s



Reponse finale: Bonjour Alice !


### Interprétation : Logging DEBUG - Anatomie d'une Invocation

**Sortie obtenue** : Les logs DEBUG révèlent chaque étape interne de SK lors de l'exécution d'une fonction.

| Timestamp | Logger | Message clé | Signification |
|-----------|--------|-------------|---------------|
| 08:07:55.813 | kernel_prompt_template | "Extracting blocks from template" | Parsing du template Jinja |
| 08:07:55.814 | kernel_function | "Function demo-greet invoking" | Début de l'invocation |
| 08:07:55.815 | kernel_function | "Function arguments: {...}" | Arguments reçus |
| 08:07:55.815 | kernel_prompt_template | "Rendering list of 3 blocks" | Rendu du template (3 blocs détectés) |
| 08:07:55.816 | kernel_prompt_template | "Rendered prompt: Dis bonjour..." | Prompt final avant envoi |
| 08:07:56.214 | open_ai_handler | "OpenAI usage: CompletionUsage(...)" | Tokens consommés (16 prompt + 3 completion) |
| 08:07:56.215 | kernel_function | "Function demo-greet succeeded" | Succès |
| 08:07:56.216 | kernel_function | "Function completed. Duration: 0.400880s" | Durée totale |

**Points clés** :
1. **3 blocs dans le template** : SK découpe le template en blocs (texte statique + variables)
2. **Tokens consommés** : 16 tokens prompt + 3 tokens completion = 19 tokens total (~0.00038$ avec GPT-4o)
3. **Latence** : 400ms totale dont ~398ms pour l'appel OpenAI (réseau + génération)
4. **Pas de cache** : `cached_tokens=0` → prompt non mis en cache par OpenAI

**Utilisation en production** :
- **Debugging** : Identifier où une fonction échoue (template, LLM, post-processing)
- **Optimisation** : Mesurer la latence de chaque composant
- **Cost tracking** : Tracker les tokens consommés par fonction
- **Audit** : Logger tous les prompts envoyés (compliance RGPD)

**Niveaux de logging recommandés** :
- **Development** : DEBUG (tout voir)
- **Staging** : INFO (invocations + erreurs)
- **Production** : WARNING (erreurs seulement) + OpenTelemetry pour les métriques

## 6. OpenTelemetry (Apercu)

Pour le monitoring en production, SK supporte OpenTelemetry.

### Architecture OpenTelemetry

```
Semantic Kernel
    |
    v
OpenTelemetry SDK
    |
    +-- Traces (appels, latence)
    |
    +-- Metrics (compteurs, histogrammes)
    |
    +-- Logs (evenements)
    |
    v
Exporters
    |
    +-- Azure Monitor
    +-- Jaeger
    +-- Prometheus
    +-- Console (debug)
```

### Exemple conceptuel (non execute)

```python
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor

# Configuration OpenTelemetry
provider = TracerProvider()
processor = SimpleSpanProcessor(ConsoleSpanExporter())
provider.add_span_processor(processor)
trace.set_tracer_provider(provider)

# SK detecte automatiquement OpenTelemetry
kernel = Kernel()
# Les traces sont emises automatiquement pour:
# - Chaque invocation de fonction
# - Chaque appel au LLM
# - Les erreurs et exceptions
```

### Metriques disponibles

| Metrique | Type | Description |
|----------|------|-------------|
| `semantic_kernel.function.invocations` | Counter | Nombre d'invocations |
| `semantic_kernel.function.duration` | Histogram | Duree des appels |
| `semantic_kernel.llm.tokens` | Counter | Tokens consommes |
| `semantic_kernel.llm.latency` | Histogram | Latence LLM |

# Conclusion

## Resume des concepts

| Concept | Description | Code cle |
|---------|-------------|----------|
| **Function Filter** | Intercepte avant/apres fonctions | `@kernel.filter(FilterTypes.FUNCTION_INVOCATION)` |
| **Prompt Filter** | Modifie le prompt | `@kernel.filter(FilterTypes.PROMPT_RENDERING)` |
| **Auto-Invoke Filter** | Controle function calling | `@kernel.filter(FilterTypes.AUTO_FUNCTION_INVOCATION)` |
| **Logging** | Debug avec logging standard | `logging.getLogger("semantic_kernel")` |
| **OpenTelemetry** | Monitoring production | Traces, metriques, logs |

## Points cles a retenir

1. **Les filtres suivent le pattern middleware** - `await next(context)` passe au suivant
2. **Plusieurs filtres peuvent etre chaines** - Ordre d'enregistrement = ordre d'execution
3. **Le contexte est mutable** - On peut modifier arguments, resultats, prompts
4. **`terminate = True`** - Stoppe l'execution (utile pour le rate limiting)
5. **OpenTelemetry pour la production** - Traces et metriques automatiques

## Exercices suggeres

1. **Filtre de cache** : Creer un filtre qui cache les resultats de fonctions
2. **Filtre d'anonymisation** : Masquer les emails/numeros dans les prompts
3. **Filtre d'approbation** : Demander confirmation avant les operations sensibles

## Pour aller plus loin

| Notebook | Contenu |
|----------|--------|
| [05-VectorStores](05-SemanticKernel-VectorStores.ipynb) | RAG avec Qdrant |
| [06-ProcessFramework](06-SemanticKernel-ProcessFramework.ipynb) | Workflows orchestres |

---

**Navigation** : [<< 03-Agents](03-SemanticKernel-Agents.ipynb) | [Index](README.md) | [05-VectorStores >>](05-SemanticKernel-VectorStores.ipynb)